In [1]:
import numpy as np
import tensorflow as tf

In [2]:
char_rdic = list('helo')
print(char_rdic)
char_dic = {w: i for i, w in enumerate(char_rdic)}
print(char_dic)
x_data = np.array([
    [1, 0, 0, 0],  # h
    [0, 1, 0, 0],  # e
    [0, 0, 1, 0],  # l
    [0, 0, 1, 0],  # l
], dtype='f')
print(x_data)
sample = [char_dic[c] for c in "hello"]
print(sample)

['h', 'e', 'l', 'o']
{'h': 0, 'e': 1, 'l': 2, 'o': 3}
[[ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  1.  0.]]
[0, 1, 2, 2, 3]


In [3]:
print len(char_dic)

4


In [15]:
# Configuration
char_vocab_size = len(char_dic)
rnn_size = char_vocab_size  # 1 hot coding (one of 4)
time_step_size = 4  # 'hell' -> predict 'ello'
batch_size = 1  # one sample

# RNN model
rnn_cell = tf.nn.rnn_cell.BasicRNNCell(rnn_size)
state = tf.zeros([batch_size, rnn_cell.state_size])
print rnn_cell.state_size 
X_split = tf.split(0, time_step_size, x_data)
print X_split
outputs, state = tf.nn.rnn(rnn_cell, X_split, state)

# logits: list of 2D Tensors of shape [batch_size x num_decoder_symbols]
# targets: list of 1D batch-sized int32 Tensors of the same length as logits.
# weights: list of 1D batch-sized float-Tensors of the same length as logits.
logits = tf.reshape(tf.concat(1, outputs), [-1, rnn_size])
targets = tf.reshape(sample[1:], [-1])
weights = tf.ones([time_step_size * batch_size])

loss = tf.nn.seq2seq.sequence_loss_by_example([logits], [targets], [weights])
cost = tf.reduce_sum(loss) / batch_size
train_op = tf.train.RMSPropOptimizer(0.01, 0.9).minimize(cost)

4
[<tf.Tensor 'split_2:0' shape=(1, 4) dtype=float32>, <tf.Tensor 'split_2:1' shape=(1, 4) dtype=float32>, <tf.Tensor 'split_2:2' shape=(1, 4) dtype=float32>, <tf.Tensor 'split_2:3' shape=(1, 4) dtype=float32>]


ValueError: Variable RNN/BasicRNNCell/Linear/Matrix already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "<ipython-input-4-9d4785a467af>", line 12, in <module>
    outputs, state = tf.nn.rnn(rnn_cell, X_split, state)
  File "/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py", line 2821, in run_ast_nodes
    if self.run_code(code, result):


In [5]:
from tensorflow.python.framework import dtypes

# Configuration
char_vocab_size = len(char_dic)
lstm_size = char_vocab_size  # 1 hot coding (one of 4)
time_step_size = 4  # 'hell' -> predict 'ello'
batch_size = 1  # one sample

# LSTM
lstm_cells = []
lstm_cell1 = tf.nn.rnn_cell.BasicLSTMCell(lstm_size, state_is_tuple=True)
lstm_cell2 = tf.nn.rnn_cell.BasicLSTMCell(lstm_size, state_is_tuple=True)
lstm_cells.append(lstm_cell1)
lstm_cells.append(lstm_cell2)
stacked_lstm = tf.nn.rnn_cell.MultiRNNCell(lstm_cells, state_is_tuple=True)

X_split = tf.split(0, time_step_size, x_data)
print(X_split)
outputs, states = tf.nn.rnn(stacked_lstm, X_split, dtype=dtypes.float32)
print(outputs)

logits = tf.reshape(tf.concat(1, outputs), [-1, lstm_size])
print(logits)
targets = tf.reshape(sample[1:], [-1])
print(targets)
weights = tf.ones([time_step_size * batch_size])

loss = tf.nn.seq2seq.sequence_loss_by_example([logits], [targets], [weights])
cost = tf.reduce_sum(loss) / batch_size
train_op = tf.train.AdamOptimizer(0.01).minimize(cost)

[<tf.Tensor 'split_1:0' shape=(1, 4) dtype=float32>, <tf.Tensor 'split_1:1' shape=(1, 4) dtype=float32>, <tf.Tensor 'split_1:2' shape=(1, 4) dtype=float32>, <tf.Tensor 'split_1:3' shape=(1, 4) dtype=float32>]
[<tf.Tensor 'RNN_1/MultiRNNCell/Cell1/BasicLSTMCell/mul_2:0' shape=(1, 4) dtype=float32>, <tf.Tensor 'RNN_1/MultiRNNCell_1/Cell1/BasicLSTMCell/mul_2:0' shape=(1, 4) dtype=float32>, <tf.Tensor 'RNN_1/MultiRNNCell_2/Cell1/BasicLSTMCell/mul_2:0' shape=(1, 4) dtype=float32>, <tf.Tensor 'RNN_1/MultiRNNCell_3/Cell1/BasicLSTMCell/mul_2:0' shape=(1, 4) dtype=float32>]
Tensor("Reshape_2:0", shape=(4, 4), dtype=float32)
Tensor("Reshape_3:0", shape=(4,), dtype=int32)


In [14]:
# print(X_split)
print(tf.shape(outputs))
print tf.shape(logits)
print tf.shape(weights)
print tf.shape(targets)


Tensor("Shape_15:0", shape=(3,), dtype=int32)
Tensor("Shape_16:0", shape=(2,), dtype=int32)
Tensor("Shape_17:0", shape=(1,), dtype=int32)
Tensor("Shape_18:0", shape=(1,), dtype=int32)
[1, 2, 2, 3]


In [5]:
# Launch the graph in a session
with tf.Session() as sess:
    # you need to initialize all variables
    tf.initialize_all_variables().run()
    for i in range(200):
        sess.run(train_op)
        result = sess.run(tf.arg_max(logits, 1))
        print("%r, %r" % (result, [char_rdic[t] for t in result]))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
array([2, 2, 2, 2]), ['l', 'l', 'l', 'l']
array([2, 2, 2, 2]), ['l', 'l', 'l', 'l']
array([2, 2, 2, 2]), ['l', 'l', 'l', 'l']
array([2, 2, 2, 2]), ['l', 'l', 'l', 'l']
array([2, 2, 2, 2]), ['l', 'l', 'l', 'l']
array([2, 2, 2, 2]), ['l', 'l', 'l', 'l']
array([2, 2, 2, 2]), ['l', 'l', 'l', 'l']
array([2, 2, 2, 2]), ['l', 'l', 'l', 'l']
array([2, 2, 2, 2]), ['l', 'l', 'l', 'l']
array([2, 2, 2, 2]), ['l', 'l', 'l', 'l']
array([2, 2, 2, 2]), ['l', 'l', 'l', 'l']
array([2, 2, 2, 2]), ['l', 'l', 'l', 'l']
array([2, 2, 2, 2]), ['l', 'l', 'l', 'l']
array([2, 2, 2, 2]), ['l', 'l', 'l', 'l']
array([2, 2, 2, 2]), ['l', 'l', 'l', 'l']
array([2, 2, 2, 2]), ['l', 'l', 'l', 'l']
array([2, 2, 2, 2]), ['l', 'l', 'l', 'l']
array([2, 2, 2, 2]), ['l', 'l', 'l', 'l']
array([2, 2, 2, 2]), ['l', 'l', 'l', 'l']
array([2, 2, 2, 2]), ['l', 'l', 'l', 'l']
array([2, 2, 2, 2]), ['l', 'l', 'l', 'l']
array([2, 2, 2, 2]), ['l', 'l', 'l', 'l']
ar